# [WSI] Ćwiczenie nr 3
### Algorytm MinMax

Mikołaj Wewiór 318407


In [1]:
import numpy as np
from src.board import Board


WAGES = [
    [3, 2, 3],
    [2, 4, 2],
    [3, 2, 3]
]
SIZE = 3

In [2]:
def evaluate_game(state: np.ndarray) -> int:
    score = 0
    size = state.shape[0]
    for row in range(size):
        for col in range(size):
            if state[row][col] == 'x':
                score -= WAGES[row][col]
            if state[row][col] == 'o':
                score += WAGES[row][col]
    return score, state

def minmax(
    board: Board,
    depth: int,
    # state: np.ndarray,
    # action: tuple, # (int, int)
    maximizing: bool,
    ) -> int:
    best_move = [None, None]
    state = board.board
    if board.round() == board.max_round() or depth == 0:
        return evaluate_game(state)[0]
    size = board.size()
    if not maximizing: # maximizing
        best = -24
        for i in range(size):
            for j in range(size):
                if state[i, j] != 'x' and state[i, j] != 'o':
                    state[i, j] = board.current_player() # potencjalny bug
                    board.board = state
                    temp_best = best
                    best = max(best, minmax(board, depth-1, maximizing=board.next_player())[0])
                    if best != temp_best:
                        best_move = [i, j]
                    state[i, j] = ""
        return best, best_move
    if maximizing: # minimizing
        best = 24
        for i in range(size):
            for j in range(size):
                # # <postać> state = np.array(tictactoe.board())
                if state[i, j] != 'x' and state[i, j] != 'o':
                    state[i, j] = board.current_player() # potencjalny bug
                    board.board = state
                    temp_best = best
                    best = min(best, minmax(board, depth-1, maximizing=board.next_player()))
                    if best != temp_best:
                        best_move = [i, j]
                    state[i, j] = ""
        return best, best_move

        # best = max(best, minmax(board, depth-1, state, action=(1, 1), not maximizing))

def findBestMove(state):
    pass

def alpha_pruning(
    state: np.ndarray,
    action: tuple, # (int, int)
    maximizing: bool,
    alpha: int,
    beta: int,
    ) -> int:
    raise NotImplementedError


In [5]:
tictactoe = Board(3)
depth = 1
tictactoe.print()
state = np.array(tictactoe.board)
size = state.size
while tictactoe.round() < size and not tictactoe.finished():
    if not tictactoe._x_player:
        tictactoe.move(True)
    else:
        value, best_move = minmax(tictactoe, depth, tictactoe._x_player)
        tictactoe.board[best_move[0]][best_move[1]] = tictactoe.current_player()
        tictactoe.update_print_board(best_move[0], best_move[1])
        tictactoe.print()
    state = np.array(tictactoe.board)
    print(evaluate_game(state)[0])



# print(tictactoe.board())
# state = np.array(tictactoe.board())



     |     |     
  1  |  2  |  3  
_____|_____|_____
     |     |     
  4  |  5  |  6  
_____|_____|_____
     |     |     
  7  |  8  |  9  
     |     |     
-----------------------------

Current player is 'o'


     |     |     
  o  |  2  |  3  
_____|_____|_____
     |     |     
  4  |  5  |  6  
_____|_____|_____
     |     |     
  7  |  8  |  9  
     |     |     
3


     |     |     
  o  |  2  |  3  
_____|_____|_____
     |     |     
  4  |  5  |  6  
_____|_____|_____
     |     |     
  7  |  8  |  9  
     |     |     


KeyboardInterrupt: 

In [4]:

# state = np.ndarray((3, 3), dtype='U7')
# stateX = state
# # print(state)
# stateX[1][1] = 'e'
# stateX[2][0] = 'E'
# state[0][0] = '2.71'
# print(state)
# print(stateX)


# def myfun(x):
#     return x, 2*x

# y = myfun(1)
# y1, y2 = myfun(1)
# print(y)
# print(y1)
# print(y2)